In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1. 讀入深度學習套件

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

### 2. 讀入數據

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=9000)

In [4]:
len(x_train)

25000

In [5]:
len(x_test)

25000

In [11]:
#第一則評論為218個字，且為正評
print(len(x_train[0]))
y_train[0]

218


1

### 3. 資料處理

In [12]:
x_train = sequence.pad_sequences(x_train, maxlen=200)
x_test = sequence.pad_sequences(x_test, maxlen=200)

### 4. step 01: 打造一個函數學習機

In [13]:
model = Sequential()

In [14]:
model.add(Embedding(9000, 128))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
#因為結果為0,1所以選用sigmoid
model.add(Dense(1, activation='sigmoid'))

In [17]:
#分類型問題用binary_crossentropy
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1152000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,283,713
Trainable params: 1,283,713
Non-trainable params: 0
_________________________________________________________________


In [19]:
#1個LSTM : 128個hidden state + 128個壓縮後的輸入 + bias
#乘上4(3個gates+1個更新狀態)、乘上128(共128個LSTM)
(128+128+1)*4*128

131584

### 5. step 02: 訓練

In [21]:
#validation: 用測試資料算誤差(不參與訓練)
model.fit(x_train, y_train, batch_size=30, epochs=5,
         validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 608s 24ms/sample - loss: 0.4574 - acc: 0.7852 - val_loss: 0.4115 - val_acc: 0.8228
Epoch 2/5
25000/25000 [==============================] - 554s 22ms/sample - loss: 0.4387 - acc: 0.8018 - val_loss: 0.4261 - val_acc: 0.8247
Epoch 3/5
25000/25000 [==============================] - 559s 22ms/sample - loss: 0.2912 - acc: 0.8828 - val_loss: 0.3334 - val_acc: 0.8599
Epoch 4/5
25000/25000 [==============================] - 517s 21ms/sample - loss: 0.2131 - acc: 0.9173 - val_loss: 0.3237 - val_acc: 0.8719
Epoch 5/5
25000/25000 [==============================] - 523s 21ms/sample - loss: 0.1576 - acc: 0.9442 - val_loss: 0.3576 - val_acc: 0.8714


### 透過調整以下設定，model正確率為0.9442
* imdb.load_data(num_words=9000)將常用字調整為10000->9000個
* 將每個sequence的maxlen調整為100->200
* 將batch_size調整為32->30，epochs調整為10->5